## Chatbot Simple con Deep Learning

**Ingeniería Electrónica**

**Inteligencia Artificial**

**19/06/2021**

In [4]:
# conda install -c anaconda nltk
import nltk
from nltk.stem.lancaster import LancasterStemmer
import numpy
# pip install tflearn
import tflearn
import tensorflow
import random
import json
# conda install -c anaconda pandas
import pandas as pd

In [8]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Francisco
[nltk_data]     Salgado\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

Comencemos por hacer nuestros datos para el chatbot. Debido a que este es un chatbot simple, no necesitamos tener datos masivos, así que lo programaremos nosotros mismos. Este debe ser un archivo .json y debe tener el mismo formato a al archivo "dataset.json" incluido en la práctica.

In [5]:
pd.read_json("dataset.json")

,dataset
0,"{'tag': 'hola', 'patterns': ['Hola', 'Que tal'..."
1,"{'tag': 'adios', 'patterns': ['Adios', 'chao',..."
2,"{'tag': 'gracias', 'patterns': ['Gracias', 'Mu..."
3,"{'tag': 'como_estas', 'patterns': ['Como estas..."


Si se desea crear una nueva categoría, la etiqueta (**tag**) es el significado y el patrón (**pattern**) es lo que el usuario diría. La respuestas (**responses**) es, claramente, las respuestas que dirá el chatbot y elegirá una de las respuestas para decir. Cuantos más datos se le proporcione, mejor será en las respuestas.

Se define el algortimo utilizado de derivación o *stemming*. En la extraccion de información y morfología lingüística, la derivación es el proceso de reducir las palabras a su forma de raíz, o base de la palabra. 
Leemos los datos en el archvio JSON:

In [6]:
stemmer = LancasterStemmer()
with open('dataset.json') as file:
    data = json.load(file)

Ahora es el momento de hacer que las listas tenga valores al extraer los datos JSON.

In [9]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['dataset']:
    for pattern in intent['patterns']:
        palabras = nltk.word_tokenize(pattern)
        words.extend(palabras)
        docs_x.append(palabras)
        docs_y.append(intent["tag"])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

Agregamos la derivación o stemming para contener todas las palabras derivadas.

In [10]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

Ahora que hemos localizado dónde están todos nuestros datos, necesitamos crear un vocabulario
.

In [11]:
training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []
    palabras = [stemmer.stem(w.lower()) for w in doc]
    
    for w in words:
        if w in palabras:
            bag.append(1)
        else:
            bag.append(0)
            
    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1
    
    training.append(bag)
    output.append(output_row)

Y ahora tomaremos los datos de entrenamiento y salida y los pondremos en arreglos.

In [12]:
training = numpy.array(training)
output = numpy.array(output)

Comenzamos a entrenar al chatbot. Entonces construimos una red neuronal estándar con 2 capas ocultas.

In [13]:
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Entrenamos al modelo y lo guardamos en el directorio actual.

In [14]:
model.fit(training, output, n_epoch = 1000, batch_size = 8, show_metric=True)
model.save("model.tflearn")

Training Step: 1999  | total loss: 0.39565 | time: 0.002s
| Adam | epoch: 1000 | loss: 0.39565 - acc: 0.9734 -- iter: 08/12
Training Step: 2000  | total loss: 0.35797 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.35797 - acc: 0.9761 -- iter: 12/12
--
INFO:tensorflow:D:\Francisco\Documents\CCTT\IA\codigo\DeepLearning\NLP\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


Se definen funciones para tener un programa de chatbot.

In [15]:
def words_list(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i,w in enumerate(words):
            if w==se:
                bag[i]=1
                
    return numpy.array(bag)

def talking():
    print("Comienza a hablar con el Chatbot (si deseas terminar el programa escribe 'salir')")
    while True:
        inp = input("Tú: ")
        if inp.lower() == "salir":
            break
        
        results = model.predict([words_list(inp, words)])
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        for tg in data['dataset']:
            if tg['tag'] == tag:
                responses = tg['responses']
        
        print(random.choice(responses))

Finalmente, iniciamos el chatbot:

In [16]:
talking()

Comienza a hablar con el Chatbot (si deseas terminar el programa escribe 'salir')
Tú: hola
Hola
Tú: qué tal
Bueno verte de nuevo
Tú: gracias
Es un placer
Tú: qué haces?
Feliz de ayudar
Tú: adios
Que tengas un lindo dia
Tú: salir


**Para revisar:**
Nick Walton, el creador de **AI Dungeon**, lo construyó  tomando el modelo de NLP GPT-2 de OpenAI, ajustándolo al elegir sus propios textos de aventura.

https://play.aidungeon.io/
    